## Notebook for making predictions with deleted constant y samples


In [1]:
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

### Reading dataset stored in other notebooks

In [2]:
sys.path.append('../')

from data_prep import dm_saver, data_manager

dms = dm_saver.DM_Saver.load('../dms')

dml = data_manager.Data_Manager()

dml.dms2dm(dms)
%store dml

Stored 'dml' (Data_Manager)


In [3]:


%store -r dml

dm = dml

## Getting data

In [4]:
print(dm.data_A.columns)

X_A_train = dm.data_A.iloc[:, 2:]
Y_A_train = dm.data_A.iloc[:, 1]

X_B_train = dm.data_B.iloc[:, 2:]
Y_B_train = dm.data_B.iloc[:, 1]

X_C_train = dm.data_C.iloc[:, 2:]
Y_C_train = dm.data_C.iloc[:, 1]

X_A_test = dm.X_test_estimated_a[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_B_test = dm.X_test_estimated_b[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_C_test = dm.X_test_estimated_c[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]

#X_A_train['direct_rad:W_lag_12']

Index(['date_forecast', 'pv_measurement', 'absolute_humidity_2m:gm3',
       'air_density_2m:kgm3', 'clear_sky_energy_1h:J', 'clear_sky_rad:W',
       'dew_or_rime:idx', 'dew_point_2m:K', 'diffuse_rad:W',
       'diffuse_rad_1h:J', 'direct_rad:W', 'direct_rad_1h:J',
       'effective_cloud_cover:p', 'elevation:m', 'fresh_snow_12h:cm',
       'fresh_snow_1h:cm', 'fresh_snow_24h:cm', 'fresh_snow_3h:cm',
       'fresh_snow_6h:cm', 'is_day:idx', 'is_in_shadow:idx',
       'msl_pressure:hPa', 'precip_5min:mm', 'precip_type_5min:idx',
       'pressure_100m:hPa', 'pressure_50m:hPa', 'prob_rime:p',
       'rain_water:kgm2', 'relative_humidity_1000hPa:p', 'sfc_pressure:hPa',
       'snow_depth:cm', 'snow_drift:idx', 'snow_melt_10min:mm',
       'snow_water:kgm2', 'sun_azimuth:d', 'sun_elevation:d',
       'super_cooled_liquid_water:kgm2', 't_1000hPa:K', 'total_cloud_cover:p',
       'visibility:m', 'wind_speed_10m:ms', 'wind_speed_u_10m:ms',
       'wind_speed_v_10m:ms', 'wind_speed_w_1000hPa:m

## Catboost models
will try with two models:
-   One for dataset A
-   One for B and C

In [5]:
catModel_a = CatBoostRegressor()
catModel_b = CatBoostRegressor()
catModel_c = CatBoostRegressor()

catModel_bc = CatBoostRegressor()

#catModel_abc = CatBoostRegressor()

In [6]:
X_BC_train = pd.concat([X_B_train, X_C_train])
Y_BC_train = pd.concat([Y_B_train, Y_C_train])

X_ABC_train = pd.concat([X_A_train, X_BC_train])
Y_ABC_train = pd.concat([Y_A_train, Y_BC_train])

In [7]:
catModel_a.fit(X_A_train,Y_A_train)
catModel_b.fit(X_B_train,Y_B_train)
catModel_c.fit(X_C_train,Y_C_train)

#catModel_bc.fit(X_BC_train,Y_BC_train)

#catModel_abc.fit(X_ABC_train,Y_ABC_train)

Learning rate set to 0.067297
0:	learn: 1247.6744460	total: 159ms	remaining: 2m 38s
1:	learn: 1186.0260545	total: 170ms	remaining: 1m 25s
2:	learn: 1130.6434933	total: 180ms	remaining: 59.8s
3:	learn: 1078.1808484	total: 188ms	remaining: 46.8s
4:	learn: 1029.7823566	total: 194ms	remaining: 38.7s
5:	learn: 985.1533261	total: 201ms	remaining: 33.2s
6:	learn: 943.6513425	total: 206ms	remaining: 29.3s
7:	learn: 906.2632481	total: 210ms	remaining: 26.1s
8:	learn: 872.3898611	total: 215ms	remaining: 23.6s
9:	learn: 840.7465563	total: 219ms	remaining: 21.7s
10:	learn: 812.1274775	total: 223ms	remaining: 20s
11:	learn: 784.7392676	total: 227ms	remaining: 18.7s
12:	learn: 760.7637750	total: 231ms	remaining: 17.6s
13:	learn: 738.6033910	total: 235ms	remaining: 16.6s
14:	learn: 718.7709395	total: 239ms	remaining: 15.7s
15:	learn: 700.9002452	total: 243ms	remaining: 14.9s
16:	learn: 684.6465070	total: 248ms	remaining: 14.3s
17:	learn: 669.7781852	total: 252ms	remaining: 13.7s
18:	learn: 655.398983

In [8]:
# Feature importance
catModel_a.get_feature_importance(prettified=True)

,Feature Id,Importances
0,direct_rad:W,26.527844
1,clear_sky_rad:W,7.307824
2,diffuse_rad:W,5.922314
3,sun_elevation:d,5.553178
4,sun_azimuth:d,5.141515
5,direct_rad_1h:J,4.751614
6,day_year,4.119622
7,direct_rad:W_lag_-23,3.660159
8,wind_speed_u_10m:ms,3.629167
9,effective_cloud_cover:p,2.705577


## Preds

In [9]:
pred_A = catModel_a.predict(X_A_test)
pred_B = catModel_b.predict(X_B_test)
pred_C = catModel_c.predict(X_C_test)

# pred_B = catModel_bc.predict(X_B_test)
# pred_C = catModel_bc.predict(X_C_test)

# pred_A = catModel_abc.predict(X_A_test)
# pred_B = catModel_abc.predict(X_B_test)
# pred_C = catModel_abc.predict(X_C_test)

In [10]:
df_A = pd.DataFrame()

df_A["predict"] = pred_A
df_A["location"] = "A"

df_B = pd.DataFrame()

df_B["predict"] = pred_B
df_B["location"] = "B"

df_C = pd.DataFrame()

df_C["predict"] = pred_C
df_C["location"] = "C"

df_mid = pd.concat([df_A, df_B], ignore_index=True)

df = pd.concat([df_mid, df_C], join="inner", ignore_index=True)

df = df.drop("location", axis=1)

df[df<0] = 0

# NAME THE FILE 
df.to_csv("./ext_preds/bestpred_old.csv")

df.shape

(2160, 1)